In [1]:
from data_preprocessing import SO_Data_preprocess, Openso_Preprocess
from utils.tools import fiilna, Save_Part_Data, Plot_History_Trend
from Indicator_Construction import Foward_Indicator, Trend_Table, Historcial_Table,Validation_Indicator_Performance
import pandas as pd 
import numpy as np

In [2]:
Region_class='CN' # CN:China (CKH是HUB廠), TW:Taiwan, EU , US
ABC_class=('A','B') # A or B or C ABC_INDICATOR.str.match("^[ABC]")
PG_class=['ISG','IAG','ICWG','ESG'] #IIOT->4['ISG','IAG','ICWG','ESG'] EIOT->3 ['ECG', 'IPSG','IDSG']

start_date="2024-08-01"
end_date="2024-10-01"
full_date_range = pd.date_range(start=start_date,end=end_date,freq='MS').normalize()   
#根據上述條件建立sql
Constrain_query="ABC_INDICATOR.str.startswith(@ABC_class)  and PG in @PG_class and SHIP_PLANT.str.startswith(@Region_class) and  year_month>=@start_date and year_month<=@end_date"


In [194]:
# Region_class+"H_1"

In [3]:
SO_Data=SO_Data_preprocess()
Used_Data=SO_Data.query(Constrain_query).reset_index(drop=True)
Used_Data=Used_Data[['COMPANY_ID','year_month','PD','MODEL','PART_NO','ORDER_QTY']]
Used_Data.head()

,COMPANY_ID,year_month,PD,MODEL,PART_NO,ORDER_QTY
0,C100432,2024-08-01,iSensing Devices,ADAM-4117,ADAM-4117-C,10
1,C302210,2024-09-01,iSensing Devices,ADAM-4117,ADAM-4117-C,6
2,C301070,2024-09-01,iSensing Devices,ADAM-4117,ADAM-4117-C,26
3,C200611,2024-09-01,iSensing Devices,ADAM-4117,ADAM-4117-C,200
4,C100052,2024-09-01,iSensing Devices,ADAM-4117,ADAM-4117-C,50


In [4]:
Used_Data.PD.unique()

array(['iSensing Devices', 'Modular IPC', 'IPC',
       'Industrial Wireless & Sensing', 'Industrial Communication',
       'Smart I/O & Communication', 'Gateway & HMI', 'Industrial Monitor',
       'Panel PC', 'UNO', 'Industrial HMI', 'Industrial Servers',
       'Machine Vision (MV)', 'Automation Control'], dtype=object)

In [5]:
Used_Data.groupby(['PD'])['ORDER_QTY'].sum().reset_index()

,PD,ORDER_QTY
0,Automation Control,175
1,Gateway & HMI,1334
2,IPC,66718
3,Industrial Communication,11189
4,Industrial HMI,49
5,Industrial Monitor,582
6,Industrial Servers,1759
7,Industrial Wireless & Sensing,463
8,Machine Vision (MV),2373
9,Modular IPC,8249


In [6]:
Openso_1=Openso_Preprocess(version_year=1).query(Constrain_query).reset_index(drop=True)
Openso_2=Openso_Preprocess(version_year=2).query(Constrain_query).reset_index(drop=True)
Openso_3=Openso_Preprocess(version_year=3).query(Constrain_query).reset_index(drop=True)

In [7]:
# PD_name='IPC' 
#PD_Material=(Used_Data.query('PD==@PD_name').groupby(['PART_NO'])['ORDER_QTY'].sum().nlargest(50)).reset_index()
PD_Material=(Used_Data.groupby(['PART_NO'])['ORDER_QTY'].sum()).reset_index()
PD_Material.PART_NO.unique()

array(['ACP-2010MB-00D', 'ACP-2010MB-25D', 'ACP-2010MB-35D',
       'ACP-2020MB-35B', 'ACP-2320MB-35D', 'ACP-4000BP-00F',
       'ACP-4000MB-00F', 'ACP-4000MB-50F', 'ACP-4020MB-50B',
       'ACP-4320MB-00C', 'ACP-4320MB-50C', 'ACP-4340MB-50B',
       'ADAM-3014-AE', 'ADAM-3016-AE', 'ADAM-3600-C2GL1A1E',
       'ADAM-3909-AE', 'ADAM-39100-BE', 'ADAM-3937-BE', 'ADAM-3951-BE',
       'ADAM-3962-AE', 'ADAM-3968-AE', 'ADAM-4015-F', 'ADAM-4017+-F',
       'ADAM-4018+-F', 'ADAM-4024-B1E', 'ADAM-4050-F', 'ADAM-4051-C',
       'ADAM-4055-C', 'ADAM-4068-C', 'ADAM-4069-B', 'ADAM-4117-C',
       'ADAM-4118-C', 'ADAM-4510S-F', 'ADAM-4520-F', 'ADAM-4561-CE',
       'ADAM-4571-CE', 'ADAM-4572-CE', 'ADAM-5000/TCP-CE',
       'ADAM-5000L/TCP-BE', 'ADAM-5017-A4E', 'ADAM-5024-A2E',
       'ADAM-5051S-AE', 'ADAM-6017-D', 'ADAM-6050-D1', 'ADAM-6052-D',
       'ADAM-6060-D1', 'ADAM-6066-D', 'ADAM-6217-B', 'ADAM-6250-B',
       'ADAM-6251-B', 'ADAM-6260-B', 'ADAM-6520-BE', 'ADAM-6520L-AE',
       'AIMB-705G2

In [8]:
# 檔案存取格式
if 'ECG' in PG_class:
    PG='EIOT'
else:
    PG='IIOT'

In [11]:
file_path = r'C:\Users\mt.yang\Desktop\SCM_Indicator\EIOT_AB_AIMB_EBC(B).csv' 

In [9]:
len(PD_Material.PART_NO.unique())

263

In [10]:
total_num,correct_num, part_number=0, 0, 0
total_up_num, total_down_num, total_tp_num, total_tn_num= 0,0,0,0
plot=False
# 料號日期條件
Material_sql=' PART_NO==@PART_NO_name '
date_column='year_month'

for PART_NO_name in PD_Material.PART_NO.unique():
    
    PART_NO_SO=Used_Data.query(Material_sql).groupby([date_column])['ORDER_QTY'].sum().reset_index()
    #open part
    PART_NO_Openso_1=Openso_1.query(Material_sql).groupby([date_column])['OPEN_QTY'].sum().reset_index()
    PART_NO_Openso_2=Openso_2.query(Material_sql).groupby([date_column])['OPEN_QTY'].sum().reset_index()
    PART_NO_Openso_3=Openso_3.query(Material_sql).groupby([date_column])['OPEN_QTY'].sum().reset_index()
    '''
    填補so & openso
    '''
    PART_NO_SO=fiilna(PART_NO_SO, full_date_range , QTY_name= 'ORDER_QTY' )
    PART_NO_Openso_1=fiilna(PART_NO_Openso_1, full_date_range , QTY_name= 'OPEN_QTY' )
    PART_NO_Openso_2=fiilna(PART_NO_Openso_2, full_date_range , QTY_name= 'OPEN_QTY' )
    PART_NO_Openso_3=fiilna(PART_NO_Openso_3, full_date_range , QTY_name= 'OPEN_QTY' )
    #用於驗證兩個月後需求量基準值
    PART_NO_SO['2_month_avg'] = PART_NO_SO['ORDER_QTY'].rolling(window=2).mean()

    #計算兩種backlog變化率的計算
    Indicator= Foward_Indicator(PART_NO_Openso_1, PART_NO_Openso_2, PART_NO_Openso_3 )
    #判定條件是否為趨勢
    Trend_Indicator =Trend_Table(Indicator)
    # 實際QTY
    Historical_info=Historcial_Table(PART_NO_SO)

    Evaluate_Table= Validation_Indicator_Performance(PART_NO_name, Trend_Indicator, Historical_info)
    # Save_Part_Data(file_path, PART_NO_SO,Trend_Indicator, PG, Region_class, PD_name, PART_NO_name ,save_file=None)

    # 總數統計
    total_num+=Evaluate_Table['Total_num'].iloc[0]
    correct_num+=(Evaluate_Table['TP_num']+Evaluate_Table['TN_num']).iloc[0]
    part_number+=1 # 料號總數統計
    total_up_num+=Evaluate_Table['Upward_num'].iloc[0]
    total_down_num+=Evaluate_Table['Downward_num'].iloc[0]
    total_tp_num+=Evaluate_Table['TP_num'].iloc[0]
    total_tn_num+=Evaluate_Table['TN_num'].iloc[0]
    '''
    畫指標和so圖 
    '''
    if plot:
        Plot_History_Trend(PART_NO_name, PART_NO_Openso_1, PART_NO_Openso_2,  PART_NO_SO, Trend_Indicator )
        print(Evaluate_Table)

d:\SCM 指標\Indicator_Construction.py:76: RuntimeWarning: invalid value encountered in scalar divide
  tp_rate=(tp_num/pos_indicator)*100
d:\SCM 指標\Indicator_Construction.py:81: RuntimeWarning: invalid value encountered in scalar divide
  tn_rate=(tn_num/neg_indicator)*100
d:\SCM 指標\Indicator_Construction.py:76: RuntimeWarning: invalid value encountered in scalar divide
  tp_rate=(tp_num/pos_indicator)*100
d:\SCM 指標\Indicator_Construction.py:81: RuntimeWarning: invalid value encountered in scalar divide
  tn_rate=(tn_num/neg_indicator)*100
d:\SCM 指標\Indicator_Construction.py:76: RuntimeWarning: invalid value encountered in scalar divide
  tp_rate=(tp_num/pos_indicator)*100
d:\SCM 指標\Indicator_Construction.py:81: RuntimeWarning: invalid value encountered in scalar divide
  tn_rate=(tn_num/neg_indicator)*100
d:\SCM 指標\Indicator_Construction.py:76: RuntimeWarning: invalid value encountered in scalar divide
  tp_rate=(tp_num/pos_indicator)*100
d:\SCM 指標\Indicator_Construction.py:81: RuntimeW

In [13]:
# 回測結果
print("Final result")
print('料號總數', part_number)
print('所有指標總數:',total_num)
print('向上指標總數', total_up_num)
print('向下指標總數', total_down_num)
print('TP rate', (total_tp_num/total_up_num)*100)
print('TN rate', (total_tn_num/total_down_num)*100)
print('Total 此PD勝率', (correct_num/total_num)*100)

Final result
料號總數 263
所有指標總數: 35
向上指標總數 26
向下指標總數 9
TP rate 19.230769230769234
TN rate 100.0
Total 此PD勝率 40.0
